### **Setting Up Metrics for Image Quality Evaluation**

**Mount Google Drive**

In [ ]:
Xfrom google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Install LPIPS Package**

Installs the LPIPS (Learned Perceptual Image Patch Similarity) library for perceptual image quality assessment

In [ ]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00


**Installing Required Libraries**

In [ ]:
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from skimage.transform import resize
from PIL import Image
import pandas as pd
import torch
import lpips
from matplotlib import pyplot as plt

**Initialize LPIPS Model**

Loads the LPIPS model with the AlexNet backbone and enables GPU acceleration for perceptual similarity computation

In [ ]:
lpips_model = lpips.LPIPS(net="alex").cuda()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 206MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), st

In [ ]:
# Paths to directories
test_dir = "/content/drive/MyDrive/ESR_GAN/test"
test_degraded_dir = "/content/drive/MyDrive/ESR_GAN/test_degraded"
test_results_dir = "/content/drive/MyDrive/ESR_GAN/test_degraded_results"

In [ ]:
test_path = "/content/drive/MyDrive/ESR_GAN/test/0002.png"
test_degraded_path = "/content/drive/MyDrive/ESR_GAN/test_degraded/0002.png"
test_restored_path = "/content/drive/MyDrive/ESR_GAN/test_degraded_results/0002_out.png"

**Load Image Function**

Loads an image from the specified filepath, converts it to RGB, and normalizes pixel values to the range [0, 1] for compatibility with metrics

In [ ]:
def load_image(filepath):
    """Load an image as a numpy array normalized to [0, 1]."""
    return np.array(Image.open(filepath).convert("RGB")) / 255.0

**Resize Image Function**

Resizes the input image to a fixed size (default: 256x256) using anti-aliasing for smoother scaling.

In [ ]:
def resize_to_fixed_size(image, target_size=(256, 256)):
    """Resize an image to a fixed target size."""
    return resize(image, target_size, anti_aliasing=True)

**Calculate SSIM with Fallback**

Computes SSIM (Structural Similarity Index) between two images. Dynamically adjusts the window size (win_size) based on image dimensions for compatibility.


In [ ]:
def calculate_ssim_with_fallback(image1, image2):
    """Calculate SSIM with a dynamically adjusted win_size."""
    h, w = image1.shape[:2]
    win_size = min(7, h, w)  # Ensure win_size doesn't exceed dimensions
    if win_size % 2 == 0:
        win_size -= 1  # Ensure win_size is odd
    return ssim(image1, image2, multichannel=True, data_range=1.0, win_size=win_size)

**Calculate LPIPS**

Calculates the LPIPS (Learned Perceptual Image Patch Similarity) score between two images. Converts images to tensors, normalizes to [-1, 1], and evaluates the similarity using the loaded LPIPS model

In [ ]:
def calculate_lpips(image1, image2):
    """Calculate LPIPS between two images."""
    image1_tensor = torch.tensor(image1.transpose(2, 0, 1) * 2 - 1, dtype=torch.float32).unsqueeze(0).cuda()
    image2_tensor = torch.tensor(image2.transpose(2, 0, 1) * 2 - 1, dtype=torch.float32).unsqueeze(0).cuda()
    with torch.no_grad():
        return lpips_model(image1_tensor, image2_tensor).item()

**Load Images**

Loads the original, degraded, and restored images as normalized numpy arrays using the load_image function

In [ ]:
# Load Images
original = load_image(test_path)
degraded = load_image(test_degraded_path)
restored = load_image(test_restored_path)

**Resize Images to Fixed Dimensions**

Resizes all images to a fixed dimension (256x256) to ensure consistent size for metric calculations

In [ ]:
# Resize All Images to Fixed Dimensions
fixed_size = (256, 256)  # Define a fixed size for all images
original_resized = resize_to_fixed_size(original, fixed_size)
degraded_resized = resize_to_fixed_size(degraded, fixed_size)
restored_resized = resize_to_fixed_size(restored, fixed_size)

**Calculate PSNR (Peak Signal-to-Noise Ratio) and SSIM (Structural Similarity Index)**

Computes PSNR to measure the fidelity of degraded and restored images relative to the original

Calculates SSIM for degraded and restored images. Handles potential errors, such as window size exceeding image dimensions

In [ ]:
# Calculate Metrics
psnr_degraded = psnr(original_resized, degraded_resized, data_range=1.0)
psnr_restored = psnr(original_resized, restored_resized, data_range=1.0)

try:
    ssim_degraded = calculate_ssim_with_fallback(original_resized, degraded_resized)
except Exception as e:
    ssim_degraded = None
    print(f"SSIM Degraded Error: {e}")

try:
    ssim_restored = calculate_ssim_with_fallback(original_resized, restored_resized)
except Exception as e:
    ssim_restored = None
    print(f"SSIM Restored Error: {e}")


SSIM Degraded Error: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.
SSIM Restored Error: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.


**Calculate LPIPS (Learned Perceptual Image Patch Similarity)**

Computes LPIPS to measure perceptual similarity between the original image and both degraded and restored versions

In [ ]:
lpips_degraded = calculate_lpips(original_resized, degraded_resized)
lpips_restored = calculate_lpips(original_resized, restored_resized)

**Displaying Image Quality Evaluation Results**

Prints the computed metrics for Peak Signal-to-Noise Ratio (PSNR), Structural Similarity Index (SSIM), and Learned Perceptual Image Patch Similarity (LPIPS). Includes fallback messages if SSIM couldn't be calculated.

In [ ]:
# Print Results
print(f"Metrics for image '0002.png':")
print(f"PSNR (Degraded): {psnr_degraded:.2f}")
print(f"PSNR (Restored): {psnr_restored:.2f}")
if ssim_degraded is not None:
    print(f"SSIM (Degraded): {ssim_degraded:.4f}")
else:
    print("SSIM (Degraded): Could not be calculated.")
if ssim_restored is not None:
    print(f"SSIM (Restored): {ssim_restored:.4f}")
else:
    print("SSIM (Restored): Could not be calculated.")
print(f"LPIPS (Degraded): {lpips_degraded:.4f}")
print(f"LPIPS (Restored): {lpips_restored:.4f}")

Metrics for image '0002.png':
PSNR (Degraded): 29.80
PSNR (Restored): 26.69
SSIM (Degraded): Could not be calculated.
SSIM (Restored): Could not be calculated.
LPIPS (Degraded): 0.1969
LPIPS (Restored): 0.1919


**SSIM Calculation with Error Handling and Normalization**

* Calculates SSIM with a manually defined win_size (3 in this case)
* Handles potential errors (e.g., ValueError) by catching exceptions and logging the error while setting ssim_value to None.
* Prints the PSNR and SSIM values for the restored image. If SSIM calculation fails, logs a fallback message

In [ ]:
# SSIM Calculation
try:
    win_size = 3  # set win_size
    ssim_value = ssim(
        orignal_resized,
        restored_resized,
        multichannel=True,
        data_range=1.0,
        win_size=win_size
    )
except ValueError as e:
    print(f"SSIM Calculation Error: {e}")
    ssim_value = None

# Normalize restored image for visualization
restored_image_resized = np.clip(restored_resized, 0, 1)  # Ensure valid pixel range

# Print Metrics
print(f"Metrics for Restored Image:")
print(f"PSNR: {psnr_value:.2f}")
if ssim_value is not None:
    print(f"SSIM: {ssim_value:.4f}")
else:
    print("SSIM could not be calculated due to image size.")


Metrics for Restored Image:
PSNR: 26.69
SSIM: 0.8684


**Batch Processing and Average Quality Metrics Calculation**


This cell processes all images in the specified directories, computes PSNR and SSIM for degraded and restored images compared to the original high-resolution images, and calculates the average metrics to evaluate overall performance

In [ ]:
# Directories for input
test_dir = "/content/drive/MyDrive/ESR_GAN/test"  # High-resolution ground truth images
test_degraded_dir = "/content/drive/MyDrive/ESR_GAN/test_degraded"  # Degraded images
test_results_dir = "/content/drive/MyDrive/ESR_GAN/test_degraded_results"  # Restored output images

# Initialize variables to store metrics
psnr_degraded_values = []
psnr_restored_values = []
ssim_degraded_values = []
ssim_restored_values = []

# Target size for resizing
target_size = (256, 256)

# Process all images in the directory
for image_name in os.listdir(test_dir):
    try:
        # File paths
        original_path = os.path.join(test_dir, image_name)
        degraded_path = os.path.join(test_degraded_dir, image_name)
        restored_path = os.path.join(test_results_dir, image_name.replace(".png", "_out.png"))

        # Load images
        original = load_image(original_path)
        degraded = load_image(degraded_path)
        restored = load_image(restored_path)

        # Resize all images to the same fixed size
        original_resized = resize_to_fixed_size(original, target_size)
        degraded_resized = resize_to_fixed_size(degraded, target_size)
        restored_resized = resize_to_fixed_size(restored, target_size)

        # PSNR Calculation
        psnr_degraded = psnr(original_resized, degraded_resized, data_range=1.0)
        psnr_restored = psnr(original_resized, restored_resized, data_range=1.0)
        psnr_degraded_values.append(psnr_degraded)
        psnr_restored_values.append(psnr_restored)

        # SSIM Calculation
        win_size = 3  # Fixed win_size
        ssim_degraded = ssim(original_resized, degraded_resized, multichannel=True, data_range=1.0, win_size=win_size)
        ssim_restored = ssim(original_resized, restored_resized, multichannel=True, data_range=1.0, win_size=win_size)
        ssim_degraded_values.append(ssim_degraded)
        ssim_restored_values.append(ssim_restored)

    except Exception as e:
        print(f"Error processing {image_name}: {e}")

# Calculate average metrics
average_psnr_degraded = np.mean(psnr_degraded_values)
average_psnr_restored = np.mean(psnr_restored_values)
average_ssim_degraded = np.mean(ssim_degraded_values)
average_ssim_restored = np.mean(ssim_restored_values)

# Print average metrics
print("Average Metrics Across All Images:")
print(f"Average PSNR (Degraded): {average_psnr_degraded:.2f}")
print(f"Average PSNR (Restored): {average_psnr_restored:.2f}")
print(f"Average SSIM (Degraded): {average_ssim_degraded:.4f}")
print(f"Average SSIM (Restored): {average_ssim_restored:.4f}")


Average Metrics Across All Images:
Average PSNR (Degraded): 25.99
Average PSNR (Restored): 26.29
Average SSIM (Degraded): 0.8055
Average SSIM (Restored): 0.8640
